# probit-регрессия: LR-тест, значимость регресии

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import chi2 # критические значения chi2

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# импорт датасета
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/loanapp.csv', na_values=(' ', '', '  '))
# импорт данных из локального файла
# df = pd.read_csv('loanapp.csv')
df

,occ,loanamt,action,msa,suffolk,appinc,typur,unit,married,dep,...,approve,mortno,mortperf,mortlat1,mortlat2,chist,multi,loanprc,thick,white
0,1,89,1,1120,0,72,0,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.754237,0.0,1
1,1,128,3,1120,0,74,0,1.0,1.0,1.0,...,0,0,1,0,0,1,0.0,0.800000,1.0,1
2,1,128,1,1120,0,84,3,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.895105,1.0,1
3,1,66,1,1120,0,36,0,1.0,1.0,0.0,...,1,0,1,0,0,0,0.0,0.600000,0.0,1
4,1,120,1,1120,0,59,8,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.895522,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,1,158,1,1120,0,96,0,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.897727,0.0,1
1985,1,35,1,1120,0,169,1,1.0,1.0,0.0,...,1,1,0,0,0,1,0.0,0.111111,0.0,1
1986,2,225,1,1120,0,49,0,2.0,1.0,0.0,...,1,0,1,0,0,1,1.0,1.000000,0.0,1
1987,1,98,1,1120,1,110,1,1.0,0.0,0.0,...,1,1,0,0,0,1,0.0,0.455814,0.0,1


## Спецификация и подгонка модели
Для датасета `loanapp`
рассморим probit-регрессию **approve на appinc, unem, male, yjob, self**

In [3]:
mod = smf.probit(formula='approve~appinc+unem+male+yjob+self', data=df) # спецификация модели
res = mod.fit() # подгонка модели
res.summary() # отчет

Optimization terminated successfully.
         Current function value: 0.371879
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                approve   No. Observations:                 1974
Model:                         Probit   Df Residuals:                     1968
Method:                           MLE   Df Model:                            5
Date:                Fri, 24 Nov 2023   Pseudo R-squ.:                0.005805
Time:                        16:33:45   Log-Likelihood:                -734.09
converged:                       True   LL-Null:                       -738.38
Covariance Type:            nonrobust   LLR p-value:                    0.1274
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2548      0.105     11.981      0.000       1.050       1.460
appinc        -0.0004      0.000     -0.988      0.323      -0.001       0.000
unem          -0.0341      0.016     -2.139      0.032      -0.065      -0.003
male           0.1063      0.093      1.147      0.251      -0.075       0.288
yjob           0.0059      0.033      0.178      0.859      -0.059       0.071
self          -0.1255      0.105     -1.194      0.233      -0.331       0.081
==============================================================================
"""

In [4]:
# Число наблюдений, по которым была оценена модель
res.nobs

1974

In [5]:
# логарифм функции правдоподобия для модели
res.llf.round(3)

-734.089

In [6]:
# логарфим функции правдоподобия для регрессии без объясняющих переменных (только на константу)
res.llnull.round(3)

-738.375

## LR-тест: значимость регресии
Тестируем значимость регрессии, т.е. гипотезу $$H_0:\beta_{appinc}=\beta_{unem}=\beta_{male}=\beta_{yjob}=\beta_{self}=0.$$

При подгонке модели LR-статистика и её P-значение вычисляется по умолчанию

## Вычислительные подробности
$LR=2(\log L-\log L_{null})$, где 
- $\log L$ – лог-правдоподобие для исходной регрессии
- $\log L_{null}$ - лог-правдоподобие для регрессии на константу

## Уровень значимости
Пусть уровень значимости $\alpha=10\%=0.1$

In [7]:
# Тестовая статистика LR-теста и её P-значение с округленим
res.llr.round(3), res.llr_pvalue.round(3)

(8.573, 0.127)

### Степени свободы для $\chi^2$-распределения 

In [8]:
res.df_model

5.0

### Критическое значение $\chi_{df}^2(\alpha)$

In [9]:
sign_level = 0.1 # уровень значимости
chi2.ppf(q=1-sign_level, df=res.df_model).round(3) 

9.236

### Значима ли регрессия? 

`Способ 1`

---
Отвергаем нулевую гипотезу при $LR_{stat}>\chi_{df}^2(\alpha)$, `регрессия значима`

Не отвергаем нулевую гипотезу при $LR_{stat}<\chi_{df}^2(\alpha)$, `регрессия незначима`

---

`Способ 2`

---
Отвергаем нулевую гипотезу при $P<\alpha$, `регрессия значима`

Не отвергаем нулевую гипотезу при $P>\alpha$, `регрессия незначима`

---

### **<span style="color:#59afe1">Вывод о значимости регрессии</span>**, используя критическое значение

`регрессия незначима` $(LR_{stat}<\chi_{df}^2(\alpha))$

### **<span style="color:#59afe1">Вывод о значимости регрессии</span>**, используя  $P$-значение

`регрессия незначима` ($P>\alpha$)